<link rel='stylesheet' href='../assets/css/main.css'/>

[<< back to main index](../README.md)

# Multiple Linear Regression Lab 6 : Akaike’s Information Criteria (AIC)

### Overview
Figure out which attributes to include using AIC

### Builds on
None

### Run time
approx. 20 minutes

### Notes



In [1]:
# initialize Spark Session
import os
import sys
top_dir = os.path.abspath(os.path.join(os.getcwd(), "../"))
if top_dir not in sys.path:
    sys.path.append(top_dir)

from init_spark import init_spark
spark = init_spark()
spark

Initializing Spark...
Spark found in :  /Users/sujee/spark
Spark config:
	 spark.app.name=TestApp
	spark.master=local[*]
	executor.memory=2g
	spark.sql.warehouse.dir=/var/folders/lp/qm_skljd2hl4xtps5vw0tdgm0000gn/T/tmpipa0tgf4
	some_property=some_value
Spark UI running on port 4040


In [2]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import RFormula


## Step 1 : House data

In [3]:
housePrices = spark.read.csv("/data/house-prices/house-sales-full.csv", header=True, inferSchema=True)
housePrices.limit(5).toPandas().T

,0,1,2,3,4
DocumentID,1,2,3,4,5
Date,9/16/14,6/16/06,1/29/07,2/25/08,3/29/13
SalePrice,280000,1000000,745000,425000,240000
PropertyID,1000102,1200013,1200019,2800016,2800024
PropertyType,Multiplex,Single Family,Single Family,Single Family,Single Family
ym,9/1/14,6/1/06,1/1/07,2/1/08,3/1/13
zhvi_px,405100,404400,425600,418400,351600
zhvi_idx,0.930836,0.929228,0.977941,0.961397,0.807904
AdjSalePrice,300805,1076162,761805,442065,297065
NbrLivingUnits,2,1,1,1,1


## Step 2: Apply an R formula for Feature Extraction

R users will be familiar with the concept of the **formula**.  The formula has a lot of features, but in its most basic form what it consists of is the following:

```
 y-variable ~ x-variable1 + xvariable2 + ....
```

basically, the y variable is the variable we are trying to predict, while the x variable(s) are the variables 
that we are using to make the prediction.  There are some complexities but that's the gist of it.

In the process, R will convert all categorical variables using one-hot encoding, and index strings.  Remember, features in spark are only allowed to be numeric (doubles).  NAs are also forbidden, so those are converted as well.

In [4]:
#lm(SalePrice ~ SqFtTotLiving + SqFtLot + Bathrooms + Bedrooms + BldgGrade + PropertyType + NbrLivingUnits + SqFtFinBasement + YrBuilt + YrRenovated + NewConstruction,
#              data = house.prices, na.action = na.omit)
    

variables = ['SqFtTotLiving', 'SqFtLot', 'Bathrooms', 'Bedrooms', 'BldgGrade', 'PropertyType',
               'NbrLivingUnits', 'SqFtFinBasement', 'YrBuilt', 'YrRenovated', 'NewConstruction']

textFormula = "SalePrice ~ SqFtTotLiving + SqFtLot + Bathrooms + Bedrooms + BldgGrade + PropertyType + \
               NbrLivingUnits + SqFtFinBasement + YrBuilt + YrRenovated + NewConstruction"

formula = RFormula(
    formula=textFormula,
    featuresCol="features",
    labelCol="label")    


featureVector = formula.fit(housePrices).transform(housePrices)
featureVector.select("features", "label").show()


+--------------------+---------+
|            features|    label|
+--------------------+---------+
|[2400.0,9373.0,3....| 280000.0|
|[3764.0,20156.0,3...|1000000.0|
|[2060.0,26036.0,1...| 745000.0|
|[3200.0,8618.0,3....| 425000.0|
|[1720.0,8620.0,1....| 240000.0|
|[930.0,1012.0,1.5...| 349900.0|
|[1750.0,34465.0,1...| 327500.0|
|[1860.0,14659.0,1...| 347000.0|
|[990.0,5324.0,1.0...| 220400.0|
|[1980.0,10585.0,2...| 437500.0|
|[840.0,12750.0,1....| 150000.0|
|[1750.0,5200.0,1....| 300000.0|
|[790.0,5240.0,1.0...| 292000.0|
|[790.0,5240.0,1.0...| 299800.0|
|[1530.0,1245.0,2....| 370000.0|
|[2120.0,7320.0,2....| 605000.0|
|[1130.0,1148.0,2....| 475000.0|
|[2620.0,3485.0,1....| 425000.0|
|[1250.0,924.0,1.5...| 357500.0|
|[1500.0,5253.0,1....| 455000.0|
+--------------------+---------+
only showing top 20 rows



## Step 2: Run MLR With All Attributes

In [5]:
glr = GeneralizedLinearRegression(family="gaussian", link="identity", maxIter=10, regParam=0.3)
lrModel = glr.fit(featureVector)

print("Coefficents:" + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

Coefficents:[170.10128030001252,0.023094538267461523,38476.26910404401,-44012.70252812829,122464.16106499206,6842.187674556212,69960.71090538551,-7206.406158130918,8.050569522709333,-3305.2407873452394,-2.7775981042860964,8089.690269342209]
Intercept: 5782921.487380644


In [6]:

# Summarize the model over the training set and print out some metrics
summary = lrModel.summary
print("Coefficient Standard Errors: " + str(summary.coefficientStandardErrors))
print("T Values: " + str(summary.tValues))
print("P Values: " + str(summary.pValues))
print("Dispersion: " + str(summary.dispersion))
print("Null Deviance: " + str(summary.nullDeviance))
print("Residual Degree Of Freedom Null: " + str(summary.residualDegreeOfFreedomNull))
print("Deviance: " + str(summary.deviance))
print("Residual Degree Of Freedom: " + str(summary.residualDegreeOfFreedom))
print("AIC: " + str(summary.aic))
print("Deviance Residuals: ")
summary.residuals().show()

Coefficient Standard Errors: [3.4654895528979646, 0.049748171659333956, 3151.1676574754106, 2047.9480513940998, 2026.1745197289538, 22643.14524271793, 23151.463884004923, 14940.292281757702, 3.9001660830952614, 66.44082047311522, 3.5197439166376037, 3993.413082376014, 133305.6697168418]
T Values: [49.08434369907935, 0.46422888514594146, 12.210162481443357, -21.491122540030993, 60.44107251006907, 0.3021747907021296, 3.0218698591116113, -0.4823470667257318, 2.064160692439081, -49.74713984278205, -0.7891477817907628, 2.0257584433336357, 43.38091170213769]
P Values: [0.0, 0.6424875043233946, 0.0, 0.0, 0.0, 0.7625211438318191, 0.002514534146429881, 0.6295633201097957, 0.03901199571881464, 0.0, 0.43003255429668874, 0.04279934126831897, 0.0]
Dispersion: 49376624674.11661
Null Deviance: 3180498185220857.5
Residual Degree Of Freedom Null: 27062
Deviance: 1335637697434854.2
Residual Degree Of Freedom: 27050
AIC: 743181.7567709518
Deviance Residuals: 
+-------------------+
|  devianceResiduals|
+

**Inspect the summary output**



## Step 3:  Run AIC calculation

We can do some parameter tuning here. In general, lower AIC is better.  By removing certain variables from the mix, we can get lower AICs and therefore a better model.

But how can we do this?  Let's programatically generate model combinations, and then run them. We're looking at all combinations of 8 variables out of 11, so that's 165 combinations.

In [ ]:
%%time

import itertools

def formulaGen(xvars, yvar):
    returnformula = yvar + " ~ "
    length = len(xvars)
    for xvar in xvars:
        returnformula = returnformula + xvar;
        length -= 1
        if (length != 0):
            returnformula = returnformula + " + "
        
    return returnformula

min_aic = summary.aic
min_model = lrModel
min_formula = textFormula

for L in range(8, len(variables)): #Find all combinations of minimum 8 variables
  for subset in itertools.combinations(variables, L):
    this_formula = formulaGen(subset, 'SalePrice')
    formula = RFormula(
        formula=this_formula,
        featuresCol="features",
        labelCol="label")
    featureVector_iter = formula.fit(housePrices).transform(housePrices)
    lr_iter = glr.fit(featureVector_iter)
    if (lr_iter.summary.aic < min_aic):
        min_aic = lr_iter.summary.aic
        min_model = lr_iter
        min_formula = this_formula
        print("New Lowest AIC found:" + str(min_aic))

print(min_formula)
print("AIC:" + str(min_aic))
# Summarize the model over the training set and print out some metrics
summary = min_model.summary
print("Coefficient Standard Errors: " + str(summary.coefficientStandardErrors))
print("T Values: " + str(summary.tValues))
print("P Values: " + str(summary.pValues))
print("Dispersion: " + str(summary.dispersion))
print("Null Deviance: " + str(summary.nullDeviance))
print("Residual Degree Of Freedom Null: " + str(summary.residualDegreeOfFreedomNull))
print("Deviance: " + str(summary.deviance))
print("Residual Degree Of Freedom: " + str(summary.residualDegreeOfFreedom))
print("AIC: " + str(summary.aic))
print("Deviance Residuals: ")
summary.residuals().show()



**Observe the formula, which attributes are included / dropped**